
## Merge datasets related to crime, subway and camera location in Mexico City


Since more datasets have been found, then they should be merged in a single dataframe after some cleaning and transformation steps. Since there is information available about the different subway lines in Mexico City, the location of the subway stations, and the location of security cameras according to the city program *Mi calle*, the intention of this notebook is to join that information with the reported crimes using the neighborhood and/or district information.

First, we import the required libraries and get the reported crimes dataset 

In [3]:
import pandas as pd
import zipfile
import numpy as np
import matplotlib.pyplot as plt

#Path
#Insert your local repo path to file 
repo_path = "INSERT YOUR LOCAL SDC-SECURITY REPO PATH HERE"
repo_path ='D:\Archivos\Social Data Challenge\sdc-security'

In [4]:
crime_data_path = "\datasets\da_victimas_completa_marzo_2023.zip"

zf = zipfile.ZipFile(repo_path+crime_data_path) 
crimes_raw = pd.read_csv(zf.open('da_victimas_completa_marzo_2023.csv'))
print(crimes_raw.shape)
crimes_raw.head()


(1038430, 22)


,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,municipio_hechos,colonia_datos,fgj_colonia_registro,latitud,longitud
0,8324429.0,2019,Enero,2019-01-04,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,...,Agosto,2018-08-29,12:00:00,12:19:00,ALVARO OBREGON,NaN,GUADALUPE INN,GUADALUPE INN,19.36125,-99.18314
1,8324430.0,2019,Enero,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2018-12-15,15:00:00,12:20:00,AZCAPOTZALCO,NaN,VICTORIA DE LAS DEMOCRACIAS,VICTORIA DE LAS DEMOCRACIAS,19.47181,-99.16458
2,8324431.0,2019,Enero,2019-01-04,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2018-12-22,15:30:00,12:23:00,COYOACAN,NaN,COPILCO EL BAJO,COPILCO UNIVERSIDAD ISSSTE,19.33797,-99.18611
3,8324435.0,2019,Enero,2019-01-04,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,...,Enero,2019-01-04,06:00:00,12:27:00,IZTACALCO,NaN,PANTITLAN V,AGRÍCOLA PANTITLAN,19.40327,-99.05983
4,8324438.0,2019,Enero,2019-01-04,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,NaN,FISICA,VICTIMA,...,Enero,2019-01-03,20:00:00,12:35:00,IZTAPALAPA,NaN,LAS AMERICAS (U HAB),PROGRESISTA,19.35480,-99.06324


## Transforming the crime dataset

In [5]:
#Change column names
crimes_raw.rename(columns = lambda x : x.lower() , inplace = True)
crimes_raw.columns = crimes_raw.columns.str.replace('ñ', 'ni')
crimes_raw.rename(columns = {"latitud":"crimen_lat", "longitud":"crimen_lon"} , inplace = True)

crimes_raw.head()

,idcarpeta,anio_inicio,mes_inicio,fechainicio,delito,categoria,sexo,edad,tipopersona,calidadjuridica,...,mes_hecho,fechahecho,horahecho,horainicio,alcaldia_hechos,municipio_hechos,colonia_datos,fgj_colonia_registro,crimen_lat,crimen_lon
0,8324429.0,2019,Enero,2019-01-04,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,...,Agosto,2018-08-29,12:00:00,12:19:00,ALVARO OBREGON,NaN,GUADALUPE INN,GUADALUPE INN,19.36125,-99.18314
1,8324430.0,2019,Enero,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2018-12-15,15:00:00,12:20:00,AZCAPOTZALCO,NaN,VICTORIA DE LAS DEMOCRACIAS,VICTORIA DE LAS DEMOCRACIAS,19.47181,-99.16458
2,8324431.0,2019,Enero,2019-01-04,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2018-12-22,15:30:00,12:23:00,COYOACAN,NaN,COPILCO EL BAJO,COPILCO UNIVERSIDAD ISSSTE,19.33797,-99.18611
3,8324435.0,2019,Enero,2019-01-04,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,...,Enero,2019-01-04,06:00:00,12:27:00,IZTACALCO,NaN,PANTITLAN V,AGRÍCOLA PANTITLAN,19.40327,-99.05983
4,8324438.0,2019,Enero,2019-01-04,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,NaN,FISICA,VICTIMA,...,Enero,2019-01-03,20:00:00,12:35:00,IZTAPALAPA,NaN,LAS AMERICAS (U HAB),PROGRESISTA,19.35480,-99.06324


### Handling null values

There are a lot of columns with null values. Since crimes dataset already has more than 1M records, erasing the records that contain null values could help to have a lighter, more accurate dataset

In [6]:
null_counts = crimes_raw.isnull().sum()
print(null_counts)

idcarpeta                     0
anio_inicio                   0
mes_inicio                    0
fechainicio                   0
delito                        0
categoria                     0
sexo                     190025
edad                     366188
tipopersona                6645
calidadjuridica               1
competencia                   0
anio_hecho                  377
mes_hecho                   377
fechahecho                  377
horahecho                   368
horainicio                    1
alcaldia_hechos               0
municipio_hechos        1028246
colonia_datos             73721
fgj_colonia_registro      50410
crimen_lat                50202
crimen_lon                50204
dtype: int64


Almost every value in column *municipio_hechos* is null, so this column will be dropped. After that, every row containing null values will be deleted. An exception is done for column *edad*, so that most of the rows are kept.


In [7]:
del crimes_raw['municipio_hechos']

In [8]:
columns_to_dropna = crimes_raw.columns.drop('edad')
crimes = crimes_raw.dropna(subset = columns_to_dropna).copy()

print('Original crime dataset shape is: {}'.format(crimes_raw.shape))
print('The shape of the new crime dataset without null values is: {}'.format(crimes.shape))

crimes.head()


Original crime dataset shape is: (1038430, 21)
The shape of the new crime dataset without null values is: (784813, 21)


,idcarpeta,anio_inicio,mes_inicio,fechainicio,delito,categoria,sexo,edad,tipopersona,calidadjuridica,...,anio_hecho,mes_hecho,fechahecho,horahecho,horainicio,alcaldia_hechos,colonia_datos,fgj_colonia_registro,crimen_lat,crimen_lon
0,8324429.0,2019,Enero,2019-01-04,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,...,2018.0,Agosto,2018-08-29,12:00:00,12:19:00,ALVARO OBREGON,GUADALUPE INN,GUADALUPE INN,19.36125,-99.18314
1,8324430.0,2019,Enero,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,...,2018.0,Diciembre,2018-12-15,15:00:00,12:20:00,AZCAPOTZALCO,VICTORIA DE LAS DEMOCRACIAS,VICTORIA DE LAS DEMOCRACIAS,19.47181,-99.16458
2,8324431.0,2019,Enero,2019-01-04,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,...,2018.0,Diciembre,2018-12-22,15:30:00,12:23:00,COYOACAN,COPILCO EL BAJO,COPILCO UNIVERSIDAD ISSSTE,19.33797,-99.18611
3,8324435.0,2019,Enero,2019-01-04,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,...,2019.0,Enero,2019-01-04,06:00:00,12:27:00,IZTACALCO,PANTITLAN V,AGRÍCOLA PANTITLAN,19.40327,-99.05983
4,8324438.0,2019,Enero,2019-01-04,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,NaN,FISICA,VICTIMA,...,2019.0,Enero,2019-01-03,20:00:00,12:35:00,IZTAPALAPA,LAS AMERICAS (U HAB),PROGRESISTA,19.35480,-99.06324


The next step is to round the numeric values to avoid unnecessary decimals.

In [9]:
crimes["idcarpeta"]  = crimes["idcarpeta"].round().astype(int)
crimes["anio_hecho"] = crimes["anio_hecho"].round().astype(int)
crimes["idcarpeta"]  = crimes["idcarpeta"].round().astype(int)

average              = crimes['edad'].mean()
crimes["edad"]       = crimes["edad"].fillna(average)
crimes["edad"]       = crimes["edad"].round().astype(int)


Converting month names to numeric values

In [10]:
month_name_to_number = {
    'enero': 1,
    'febrero': 2,
    'marzo': 3,
    'abril': 4,
    'mayo': 5,
    'junio': 6,
    'julio': 7,
    'agosto': 8,
    'septiembre': 9,
    'octubre': 10,
    'noviembre': 11,
    'diciembre': 12
}

crimes["mes_inicio"] = crimes["mes_inicio"].str.lower().map(month_name_to_number) 
crimes["mes_hecho"] = crimes["mes_hecho"].str.lower().map(month_name_to_number) 

crimes.head()

,idcarpeta,anio_inicio,mes_inicio,fechainicio,delito,categoria,sexo,edad,tipopersona,calidadjuridica,...,anio_hecho,mes_hecho,fechahecho,horahecho,horainicio,alcaldia_hechos,colonia_datos,fgj_colonia_registro,crimen_lat,crimen_lon
0,8324429,2019,1,2019-01-04,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62,FISICA,OFENDIDO,...,2018,8,2018-08-29,12:00:00,12:19:00,ALVARO OBREGON,GUADALUPE INN,GUADALUPE INN,19.36125,-99.18314
1,8324430,2019,1,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38,FISICA,VICTIMA Y DENUNCIANTE,...,2018,12,2018-12-15,15:00:00,12:20:00,AZCAPOTZALCO,VICTORIA DE LAS DEMOCRACIAS,VICTORIA DE LAS DEMOCRACIAS,19.47181,-99.16458
2,8324431,2019,1,2019-01-04,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42,FISICA,VICTIMA Y DENUNCIANTE,...,2018,12,2018-12-22,15:30:00,12:23:00,COYOACAN,COPILCO EL BAJO,COPILCO UNIVERSIDAD ISSSTE,19.33797,-99.18611
3,8324435,2019,1,2019-01-04,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35,FISICA,VICTIMA Y DENUNCIANTE,...,2019,1,2019-01-04,06:00:00,12:27:00,IZTACALCO,PANTITLAN V,AGRÍCOLA PANTITLAN,19.40327,-99.05983
4,8324438,2019,1,2019-01-04,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,39,FISICA,VICTIMA,...,2019,1,2019-01-03,20:00:00,12:35:00,IZTAPALAPA,LAS AMERICAS (U HAB),PROGRESISTA,19.35480,-99.06324


In [11]:
crimes[crimes["colonia_datos"] != crimes["fgj_colonia_registro"]].shape

(610309, 21)

There are a lot of records where *colonia_datos* and *fgj_colonia_registro* do not match with each other. The reason is that *colonia* names have variations between data sources. Column *fgj_colonia_registro* contains more generic names, so this will be used. The Column *colonia_datos* might help with the homologation process. If it doesnt, it will be just deleted.

In [12]:
crimes.reset_index(drop=True, inplace =True)


### Checking for duplicates

The following lines suggests that there might be a lot of reasons why there are duplicates on the *crimes.idcarpeta* column. For example, the same *idcarpeta* crime can have multiple victims. And those victims can have the same age and gender. For 

In [13]:
duplicates = crimes['idcarpeta'].duplicated(keep=False)
duplicated_rows=crimes[duplicates]
print("Number of duplicated idcarpetas: {}".format(duplicated_rows.shape[0]))
duplicated_rows[['idcarpeta','delito','categoria','sexo','edad','tipopersona','fgj_colonia_registro','alcaldia_hechos']].sort_values(by=['idcarpeta']).head(15)

Number of duplicated idcarpetas: 55679


,idcarpeta,delito,categoria,sexo,edad,tipopersona,fgj_colonia_registro,alcaldia_hechos
95152,8322427,ROBO A NEGOCIO CON VIOLENCIA,ROBO A NEGOCIO CON VIOLENCIA,Femenino,43,FISICA,EL RETOÑO,IZTAPALAPA
95153,8322427,ROBO A NEGOCIO CON VIOLENCIA,ROBO A NEGOCIO CON VIOLENCIA,Masculino,55,FISICA,EL RETOÑO,IZTAPALAPA
49556,8322439,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,Femenino,45,FISICA,EL ERMITAÑO,LA MAGDALENA CONTRERAS
106332,8322439,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,Masculino,15,FISICA,EL ERMITAÑO,LA MAGDALENA CONTRERAS
49578,8322533,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Masculino,1,FISICA,SAN MARCOS NORTE,XOCHIMILCO
95184,8322533,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Femenino,4,FISICA,SAN MARCOS NORTE,XOCHIMILCO
106354,8322550,HOMICIDIO POR ARMA DE FUEGO,HOMICIDIO DOLOSO,Masculino,26,FISICA,MOSCO CHINAMPA,IZTACALCO
95192,8322550,HOMICIDIO POR ARMA DE FUEGO,HOMICIDIO DOLOSO,Masculino,30,FISICA,MOSCO CHINAMPA,IZTACALCO
95198,8322572,DAÑO EN PROPIEDAD AJENA CULPOSA POR TRÁNSITO V...,DELITO DE BAJO IMPACTO,Femenino,39,FISICA,LAS AGUILAS 3ER PARQUE,ALVARO OBREGON
106364,8322572,DAÑO EN PROPIEDAD AJENA CULPOSA POR TRÁNSITO V...,DELITO DE BAJO IMPACTO,Femenino,39,FISICA,LAS AGUILAS 3ER PARQUE,ALVARO OBREGON


Checking duplicates in more columns: 

In [14]:
#Checking row duplicates in every single columns
duplicates = crimes[crimes.columns].duplicated(keep=False)
#duplicates = crimes[crimes.columns].duplicated(keep='last')
#duplicated_rows=crimes[duplicates].copy() 
duplicated_rows=crimes[duplicates]
print(duplicated_rows[['idcarpeta','delito','categoria','sexo','edad','tipopersona','crimen_lat','crimen_lon']].sort_values(by=['idcarpeta']).shape)
duplicated_rows["duplicates"]=duplicates
duplicated_rows[['idcarpeta','delito','categoria','sexo','edad','tipopersona','crimen_lat','crimen_lon','duplicates']].head(30)#.sort_values(by=['idcarpeta']).head(30)
crimes.head(100)

(8686, 8)


C:\Users\Marco .LAPTOP-8I11FSGG\AppData\Local\Temp\ipykernel_33564\3495241093.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicated_rows["duplicates"]=duplicates


,idcarpeta,anio_inicio,mes_inicio,fechainicio,delito,categoria,sexo,edad,tipopersona,calidadjuridica,...,anio_hecho,mes_hecho,fechahecho,horahecho,horainicio,alcaldia_hechos,colonia_datos,fgj_colonia_registro,crimen_lat,crimen_lon
0,8324429,2019,1,2019-01-04,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62,FISICA,OFENDIDO,...,2018,8,2018-08-29,12:00:00,12:19:00,ALVARO OBREGON,GUADALUPE INN,GUADALUPE INN,19.36125,-99.18314
1,8324430,2019,1,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38,FISICA,VICTIMA Y DENUNCIANTE,...,2018,12,2018-12-15,15:00:00,12:20:00,AZCAPOTZALCO,VICTORIA DE LAS DEMOCRACIAS,VICTORIA DE LAS DEMOCRACIAS,19.47181,-99.16458
2,8324431,2019,1,2019-01-04,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42,FISICA,VICTIMA Y DENUNCIANTE,...,2018,12,2018-12-22,15:30:00,12:23:00,COYOACAN,COPILCO EL BAJO,COPILCO UNIVERSIDAD ISSSTE,19.33797,-99.18611
3,8324435,2019,1,2019-01-04,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35,FISICA,VICTIMA Y DENUNCIANTE,...,2019,1,2019-01-04,06:00:00,12:27:00,IZTACALCO,PANTITLAN V,AGRÍCOLA PANTITLAN,19.40327,-99.05983
4,8324438,2019,1,2019-01-04,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,39,FISICA,VICTIMA,...,2019,1,2019-01-03,20:00:00,12:35:00,IZTAPALAPA,LAS AMERICAS (U HAB),PROGRESISTA,19.35480,-99.06324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,8324792,2019,1,2019-01-04,ROBO DE VEHICULO DE PEDALES,DELITO DE BAJO IMPACTO,Masculino,39,MORAL,OFENDIDO,...,2019,1,2019-01-04,17:00:00,18:19:00,AZCAPOTZALCO,EL ROSARIO C (U HAB),EL ROSARIO,19.50326,-99.20375
96,8324805,2019,1,2019-01-04,ROBO A TRANSEUNTE EN VIA PUBLICA SIN VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,Femenino,42,FISICA,VICTIMA Y DENUNCIANTE,...,2018,12,2018-12-27,21:30:00,18:34:00,CUAUHTEMOC,CENTRO VI,CENTRO,19.42623,-99.13303
97,8324808,2019,1,2019-01-04,ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA,DELITO DE BAJO IMPACTO,Masculino,34,FISICA,VICTIMA Y DENUNCIANTE,...,2018,10,2018-10-19,23:00:00,18:41:00,TLALPAN,VILLA LAZARO CARDENAS,VILLA LÁZARO CÁRDENAS,19.29550,-99.14017
98,8324813,2019,1,2019-01-04,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,Masculino,52,FISICA,VICTIMA Y DENUNCIANTE,...,2018,12,2018-12-24,12:00:00,18:45:00,LA MAGDALENA CONTRERAS,HUAYATLA,HUAYATLA,19.30641,-99.26473


Since there columns on this dataset do not have unique value constraints, the next lines will only erase the duplicates who match on every single column: a double entry of the same record is more possible when we look at the indexes and realize that a lot of those duplicate contain non consecutive indexes. Those should be dropped, and the ones with consecutive indexes, kept, just to allow the chances of having more than one victim of the same crime with the same age and gender.

In [15]:
for col in ["is_duplicate", "idcarpeta_on_next_row"]:
    if col in crimes.columns:
        del crimes[col]  

In [16]:
crimes["is_duplicate"] = crimes.duplicated(keep='last')
crimes['idcarpeta_on_next_row']  = (crimes['idcarpeta'] == crimes['idcarpeta'].shift(-1))



In [17]:
crimes[crimes['is_duplicate'] == True]


,idcarpeta,anio_inicio,mes_inicio,fechainicio,delito,categoria,sexo,edad,tipopersona,calidadjuridica,...,fechahecho,horahecho,horainicio,alcaldia_hechos,colonia_datos,fgj_colonia_registro,crimen_lat,crimen_lon,is_duplicate,idcarpeta_on_next_row
7518,8454529,2019,6,2019-06-18,HOMICIDIO POR ARMA DE FUEGO,HOMICIDIO DOLOSO,Masculino,39,FISICA,CADAVER,...,2019-06-18,17:00:00,17:19:45,TLALPAN,HEROES DE PADIERNA II,HÉROES DE PADIERNA,19.282140,-99.222300,True,False
8897,8437644,2019,5,2019-05-28,HOMICIDIO POR ARMA DE FUEGO,HOMICIDIO DOLOSO,Masculino,39,FISICA,CADAVER,...,2019-05-27,21:30:00,03:12:00,IZTAPALAPA,LOMAS DE ZARAGOZA,LOMAS DE ZARAGOZA,19.349500,-98.996640,True,True
31312,8461163,2019,6,2019-06-26,DAÑO EN PROPIEDAD AJENA INTENCIONAL A NEGOCIO,DELITO DE BAJO IMPACTO,Masculino,39,FISICA,OFENDIDO,...,2019-06-26,13:30:00,20:42:18,GUSTAVO A. MADERO,LA PURISIMA TICOMAN (BARR),LA PURÍSIMA TICOMÁN,19.516680,-99.138890,True,False
32099,8509270,2019,8,2019-08-29,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,39,FISICA,OFENDIDO,...,2019-08-27,16:00:00,14:46:27,ALVARO OBREGON,LAS AGUILAS (AMPL),Las Aguilas (AOB-3),19.348830,-99.217930,True,False
41709,8400645,2019,4,2019-04-09,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,DELITO DE BAJO IMPACTO,Femenino,39,FISICA,LESIONADO,...,2019-04-09,13:30:00,14:23:39,COYOACAN,ROMERO DE TERREROS (FRACC),ROMERO DE TERREROS,19.345800,-99.175320,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781638,9425294,2023,3,2023-03-27,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Femenino,39,FISICA,VICTIMA,...,2023-03-07,12:00:00,14:44:00,LA MAGDALENA CONTRERAS,BARRANCA SECA,BARRIO SAN FRANCISCO,19.307886,-99.241243,True,True
782773,9426831,2023,3,2023-03-29,ROBO A PASAJERO A BORDO DE PESERO COLECTIVO CO...,ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN ...,Femenino,39,FISICA,VICTIMA Y DENUNCIANTE,...,2023-03-29,00:20:00,01:51:00,TLAHUAC,GRANJAS CABRERA,LOS OLIVOS,19.305349,-99.062685,True,True
783534,9427885,2023,3,2023-03-30,DAÑO EN PROPIEDAD AJENA INTENCIONAL,DELITO DE BAJO IMPACTO,Masculino,39,FISICA,VICTIMA Y DENUNCIANTE,...,2023-03-30,09:15:00,11:40:00,XOCHIMILCO,VILLA XOCHIMILCO (U HAB),LA NORIA,19.268043,-99.125996,True,True
784216,9428780,2023,3,2023-03-31,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Masculino,39,FISICA,VICTIMA,...,2023-03-30,22:00:00,10:20:00,ALVARO OBREGON,BELLA VISTA,BELLAVISTA,19.395977,-99.190337,True,True


In [18]:
crimes2 = crimes[ ~ (crimes["is_duplicate"]) | (crimes["idcarpeta_on_next_row"] == True)].copy()

In [19]:
print('Dataframe before dropping duplicates: {}'.format(crimes.shape))
print('Dataframe after dropping duplicates: {}'.format(crimes2.shape))

Dataframe before dropping duplicates: (784813, 23)
Dataframe after dropping duplicates: (784802, 23)


Only 11 rows were deleted, since it seems most of the duplicate records were inserted consecutively. There is still uncertainty about the reason of the consecutive duplication of records, but we will keep the assumption that this is about same crime with more than ione victim with the same age and gender on the same location.

There are some outlier values in 'edad' colum. Those will be replaced by the average age, unless they have information about deaths not related with crime reasons.


In [20]:
crimes2[crimes2["edad"] > 100 ]

,idcarpeta,anio_inicio,mes_inicio,fechainicio,delito,categoria,sexo,edad,tipopersona,calidadjuridica,...,fechahecho,horahecho,horainicio,alcaldia_hechos,colonia_datos,fgj_colonia_registro,crimen_lat,crimen_lon,is_duplicate,idcarpeta_on_next_row
7727,8432308,2019,5,2019-05-21,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,Masculino,258,FISICA,VICTIMA Y DENUNCIANTE,...,2019-05-20,18:15:00,14:42:00,VENUSTIANO CARRANZA,CENTRO I,ZONA CENTRO,19.430840,-99.124680,False,False
72665,8582377,2019,12,2019-12-04,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,Masculino,369,FISICA,VICTIMA Y DENUNCIANTE,...,2019-12-04,00:10:00,01:49:10,IZTAPALAPA,PASEOS DE CHURUBUSCO,CENTRAL DE ABASTO,19.382520,-99.087010,False,False
87863,8407714,2019,4,2019-04-18,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,Masculino,111,FISICA,VICTIMA Y DENUNCIANTE,...,2019-04-15,07:30:00,10:07:53,VENUSTIANO CARRANZA,ARTES GRAFICAS,ARTES GRAFICAS,19.411140,-99.124840,False,False
171262,8530411,2019,9,2019-09-26,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Masculino,104,FISICA,VICTIMA,...,2019-06-28,12:00:00,10:21:27,GUSTAVO A. MADERO,PROVIDENCIA II,PROVIDENCIA,19.481600,-99.067450,False,False
287347,8725367,2020,7,2020-07-24,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Femenino,103,FISICA,VICTIMA Y DENUNCIANTE,...,2020-07-23,12:45:00,14:38:20,IZTAPALAPA,CONSTITUCION DE 1917 I,CONSTITUCIÓN DE 1917,19.351295,-99.066117,False,False
317655,8767258,2020,9,2020-09-29,PERDIDA DE LA VIDA POR OTRAS CAUSAS,HECHO NO DELICTIVO,Masculino,114,FISICA,CADAVER,...,2020-09-28,17:30:00,14:35:54,GUSTAVO A. MADERO,PRADOS DE CUAUTEPEC,PRADOS DE CUAUTEPEC,19.560390,-99.140933,False,False
345526,8805407,2020,11,2020-11-24,PERDIDA DE LA VIDA POR ENFERMEDAD,HECHO NO DELICTIVO,Masculino,110,FISICA,CADAVER,...,2020-11-24,11:00:00,13:06:43,TLAHUAC,AGRICOLA METROPOLITANA,VILLA CENTRO AMERICANA,19.287869,-99.052495,False,False
403290,8883834,2021,3,2021-03-28,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Femenino,120,FISICA,VICTIMA Y DENUNCIANTE,...,2021-03-21,22:30:00,14:19:00,IZTAPALAPA,XALPA I,XALPA,19.334401,-99.015091,False,False
421776,8917704,2021,5,2021-05-01,ROBO A PASAJERO / CONDUCTOR DE VEHICULO CON VI...,DELITO DE BAJO IMPACTO,Masculino,101,FISICA,VICTIMA Y DENUNCIANTE,...,2021-05-01,10:30:00,14:40:00,XOCHIMILCO,SANTA MARIA TEPEPAN (PBLO),SANTA MARÍA TEPEPAN,19.265889,-99.138503,False,False
444594,8950061,2021,6,2021-06-13,AMENAZAS,DELITO DE BAJO IMPACTO,Masculino,101,FISICA,VICTIMA Y DENUNCIANTE,...,2021-05-05,10:00:00,12:49:00,TLAHUAC,MIGUEL HIDALGO OTE,MIGUEL HIDALGO,19.289929,-99.042323,False,False


In [21]:
# Conditional statement of the rows we want to change
mask  = (crimes2["categoria"] != 'HECHO NO DELICTIVO') & (crimes2["edad"] > 100)

#Calculating average age
average_age = np.average(crimes2['edad'])

crimes2.loc[mask,'edad'] = average_age
crimes2["edad"] = crimes2["edad"].round().astype(int)

#Validate that only non crime related deaths are related to people older than 100 years
crimes2[crimes2["edad"] > 100 ]

,idcarpeta,anio_inicio,mes_inicio,fechainicio,delito,categoria,sexo,edad,tipopersona,calidadjuridica,...,fechahecho,horahecho,horainicio,alcaldia_hechos,colonia_datos,fgj_colonia_registro,crimen_lat,crimen_lon,is_duplicate,idcarpeta_on_next_row
317655,8767258,2020,9,2020-09-29,PERDIDA DE LA VIDA POR OTRAS CAUSAS,HECHO NO DELICTIVO,Masculino,114,FISICA,CADAVER,...,2020-09-28,17:30:00,14:35:54,GUSTAVO A. MADERO,PRADOS DE CUAUTEPEC,PRADOS DE CUAUTEPEC,19.560390,-99.140933,False,False
345526,8805407,2020,11,2020-11-24,PERDIDA DE LA VIDA POR ENFERMEDAD,HECHO NO DELICTIVO,Masculino,110,FISICA,CADAVER,...,2020-11-24,11:00:00,13:06:43,TLAHUAC,AGRICOLA METROPOLITANA,VILLA CENTRO AMERICANA,19.287869,-99.052495,False,False


Finally, some column names will be replaced for a simpler option

In [22]:
for col in ["is_duplicate", "idcarpeta_on_next_row"]:
    if col in crimes2.columns:
        del crimes2[col]  
        
crimes2.rename(columns = {"alcaldia_hechos":"alcaldia", "fgj_colonia_registro":"colonia", "colonia_datos":"colonia_alt" } , inplace = True)
crimes2.columns = crimes2.columns.str.replace('_inicio', '_denuncia')
crimes2.columns = crimes2.columns.str.replace('inicio', '_denuncia')
crimes2.columns = crimes2.columns.str.replace('ahecho', 'a_hecho')

In [23]:
crimes2 = crimes2[['idcarpeta', 'delito', 'categoria', 'alcaldia', 'colonia', 'sexo', 'edad', 'tipopersona', 'calidadjuridica',
                  'anio_denuncia', 'mes_denuncia', 'fecha_denuncia', 'hora_denuncia','competencia',   
                  'anio_hecho', 'mes_hecho', 'fecha_hecho', 'hora_hecho','colonia_alt', 'crimen_lat','crimen_lon']]

In [24]:
print(type(crimes2["fecha_denuncia"][0]))
print(type(crimes2["fecha_hecho"][0]))
print(type(crimes2["anio_hecho"][0]))

<class 'str'>
<class 'str'>
<class 'numpy.int32'>


## Checking issues with date related data

In [25]:
crimes2.anio_hecho.unique()


array([2018, 2019, 2010, 2017, 2016, 2014, 2000, 2012, 1996, 2011, 2006,
       2015, 2013, 1972, 1983, 2003, 1971, 2009, 2004, 1998, 2008, 2005,
       1987, 2007, 1968, 1993, 1995, 2001, 1981, 1999, 2002, 1986, 1992,
       1994, 1989, 1991, 1990, 1974, 1997, 1984, 1985, 1964, 1969, 1982,
       1975, 2020, 1950, 1988, 1979, 1962, 1952, 1980, 2021, 1970, 1955,
       1976, 1963, 1977, 1967, 1958, 1917, 1966, 2022, 1978,  222, 2023])

Data related to crimes commited many years ago is not relevant to this analysis, since it doesn't reveal the recent crime behaviour. So data before 2015 will be removed from this dataset

In [26]:
crimes2['fecha_hecho'].replace('0222-03-20', '03/20/2022', inplace = True)
crimes2['anio_hecho'].replace(222, 2022, inplace = True)


In [27]:
crimes_final= crimes2[ crimes2['anio_hecho'] > 2014].copy()

print('Dataframe before dropping old cases: {}'.format(crimes2.shape))
print('Dataframe after dropping old cases: {}'.format(crimes_final.shape))


Dataframe before dropping old cases: (784802, 21)
Dataframe after dropping old cases: (781226, 21)


In [28]:
crimes_final["hecho_time"]    = pd.to_datetime(crimes_final['fecha_hecho']    + ' ' + crimes_final['hora_hecho']     , infer_datetime_format=True)
crimes_final["denuncia_time"] = pd.to_datetime(crimes_final['fecha_denuncia'] + ' ' + crimes_final['hora_denuncia']  , infer_datetime_format=True)


In [29]:
crimes_final.head()

,idcarpeta,delito,categoria,alcaldia,colonia,sexo,edad,tipopersona,calidadjuridica,anio_denuncia,...,competencia,anio_hecho,mes_hecho,fecha_hecho,hora_hecho,colonia_alt,crimen_lat,crimen_lon,hecho_time,denuncia_time
0,8324429,FRAUDE,DELITO DE BAJO IMPACTO,ALVARO OBREGON,GUADALUPE INN,Masculino,62,FISICA,OFENDIDO,2019,...,FUERO COMUN,2018,8,2018-08-29,12:00:00,GUADALUPE INN,19.36125,-99.18314,2018-08-29 12:00:00,2019-01-04 12:19:00
1,8324430,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,AZCAPOTZALCO,VICTORIA DE LAS DEMOCRACIAS,Femenino,38,FISICA,VICTIMA Y DENUNCIANTE,2019,...,FUERO COMUN,2018,12,2018-12-15,15:00:00,VICTORIA DE LAS DEMOCRACIAS,19.47181,-99.16458,2018-12-15 15:00:00,2019-01-04 12:20:00
2,8324431,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,COYOACAN,COPILCO UNIVERSIDAD ISSSTE,Masculino,42,FISICA,VICTIMA Y DENUNCIANTE,2019,...,FUERO COMUN,2018,12,2018-12-22,15:30:00,COPILCO EL BAJO,19.33797,-99.18611,2018-12-22 15:30:00,2019-01-04 12:23:00
3,8324435,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,IZTACALCO,AGRÍCOLA PANTITLAN,Masculino,35,FISICA,VICTIMA Y DENUNCIANTE,2019,...,FUERO COMUN,2019,1,2019-01-04,06:00:00,PANTITLAN V,19.40327,-99.05983,2019-01-04 06:00:00,2019-01-04 12:27:00
4,8324438,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,IZTAPALAPA,PROGRESISTA,Masculino,39,FISICA,VICTIMA,2019,...,FUERO COMUN,2019,1,2019-01-03,20:00:00,LAS AMERICAS (U HAB),19.35480,-99.06324,2019-01-03 20:00:00,2019-01-04 12:35:00


## Exporting the dataset
The resulting dataset after the transformations will be exported to a csv

In [30]:
#crimes_final.to_csv(repo_path+'/datasets/crimes.csv', index=False)

In [31]:
crimes_final.to_csv('../datasets/crimes_clean.csv.zip', compression = 'zip', index=False)

## Getting the new datasets

In [ ]:
!pip install xlrd

In [ ]:
# Insert the path of the dataset in your local machine
metro_stations_path = "\datasets\metro\metro_cdmx_estaciones.xls"
stations_raw = pd.read_excel(repo_path+metro_stations_path)



## Transforming the new datasets
### Metro stations dataset

The *Sistema* column has the same value in all rows, so it will be deleted. Also, some column names are going to be changed:

In [ ]:
stations_raw.head()

In [ ]:
stations_raw.rename(columns = lambda x : x.lower() , inplace = True)
stations_raw.rename(columns = {"a_o":"year", "fid":"id"} , inplace = True)

del stations_raw['sistema']


There is a column called *geometry* that contains, in string format, longitude and latitude. The numeric values from this string will be extracted and stored in two new columns

In [ ]:

# Regular expression pattern to extract numeric values
pattern = r"\((-?\d+\.\d+) (-?\d+\.\d+)\)"

import re

def extract_coordinates(point_str):
    matches = re.findall(pattern, point_str)
    if matches:
        return pd.Series(matches[0], index=['station_lon', 'station_lat'])
    return pd.Series([None, None], index=['station_lon', 'station_lat'])

stations_raw[['station_lon', 'station_lat']] = stations_raw['geometry'].apply(extract_coordinates)



The columns *geometry* and *cve_est* will be dropped because they are redundant . *id* column will be converted to a numeric value

In [ ]:
del stations_raw['geometry']
del stations_raw['cve_est']

The categorical values in column id will be replaced by a numeric value by erasing everything but the number inside the column

In [ ]:
stations_raw['id'] = stations_raw.id.str.replace('cdmx_estaciones_metro.', '')
stations_raw['id'] = pd.to_numeric(stations_raw['id'])

In [ ]:
null_counts = stations_raw.isnull().sum()
print(null_counts)

No null values are present on this dataset, so there won't be any deleted rows.

In [ ]:
stations = stations_raw.loc[:,['id','nombre','linea','est','cve_eod17','tipo','alcaldias','year','station_lat','station_lon']].copy()
stations.head()


### Cams dataset


In [ ]:
cams_path = "\datasets\mi-calle_camaras\programa-mi-calle-shapes.zip"
zf = zipfile.ZipFile(repo_path+cams_path) 
cams_raw = pd.read_csv(zf.open('programa-mi-calle-shapes.csv'))
print(cams_raw.shape)
cams_raw.head()

The cams dataset contains a column with the centroid of the *colonia* or neighborhood. The numeric values will be extracted and saved in two new columns. Also, the values from the categorical variable *prioritari* will be replaced by 1 or 0. 

In [ ]:
cams_raw.isnull().sum()

There are only 17 null values in the neighborhood location data. But those rows will be deleted.

In [ ]:
cams_raw = cams_raw.dropna()

cams_raw[['colonia_lat', 'colonia_lon']] = cams_raw['geo_point_2d'].str.split(',', expand=True)
cams_raw['colonia_lat'] = pd.to_numeric(cams_raw['colonia_lat'])
cams_raw['colonia_lon'] = pd.to_numeric(cams_raw['colonia_lon'])

del cams_raw['geo_point_2d']

cams_raw['prioritari'] = cams_raw['prioritari'].replace({'Si': 1, 'No': 0})



In [ ]:
cams = cams_raw.loc[:,['id','alcaldia','colonia','totalinsta','prioritari','geo_shape','colonia_lat','colonia_lon']].copy()
cams.head()

### Checking for duplicates
It turns out that there are data duplicates on the datasets. That has to do with errors during collection or updates that were done by adding a new record without erasing the previous one. In the case of the cams dataset, there are inconsistencies on the number of installed cams for the same *colonia* in the same district or *alcaldia*. Cleaning is required on this stage before future merges.

In [ ]:
duplicates = cams[['colonia', 'alcaldia']].duplicated(keep=False)
duplicated_rows=cams[duplicates]
duplicated_rows.sort_values(by=['alcaldia','colonia']).head(15)

Since there is uncertainty about which record to keep (for instance, look at colonia Santa Fe example on the dataframe above), the strategy will be keeping the last option: the assumption is that it will contain the most accurate information.

In [ ]:
print('Shape of cams dataset before dropping duplicates: ', cams.shape)

cams = cams.drop_duplicates(['colonia', 'alcaldia'], keep ='last')

print('Shape of cams dataset after dropping duplicates: ', cams.shape)

#Verifying that the duplicates are gone
duplicates = cams[['colonia', 'alcaldia']].duplicated(keep=False)
duplicated_rows=cams[duplicates]
duplicated_rows.sort_values(by=['alcaldia','colonia']).head(15)

In [ ]:
duplicates = cams[['colonia', 'alcaldia', 'totalinsta']].duplicated(keep=False)
duplicated_rows=cams[duplicates]
duplicated_rows.sort_values(by=['alcaldia','colonia']).head(15)

## Merging stage
The **DataFrame.merge** will be used to merge dataframes **crimes_final** and **cams**. *crimes_last.colonia* can be compared with the *cams.colonia* column, and the same can be done with *crimes-final.alcaldia* and *cams.alcaldia*. Some columns will be dropped or their order will be changed.


In [ ]:
print('Crimes dataset shape is ', crimes_final.shape)
print('Cams dataset shape is ', cams.shape)

print(crimes.isnull().sum())
print(cams.isnull().sum())


In [ ]:
merged_df = pd.merge(crimes_final, cams, on=['alcaldia','colonia'], how='left')


In [ ]:
merged_df

In [ ]:
# Create a new column to indicate whether to use 'colonia_alt'
merged_df2 = pd.merge(crimes_final, cams, left_on=['alcaldia','colonia_alt'],right_on=['alcaldia','colonia'], how='left')


In [ ]:
print(merged_df.isnull().sum())
print(merged_df2.isnull().sum())


In [ ]:
merge_final = merged_df.combine_first(merged_df2)


In [ ]:
print(merge_final.isnull().sum())


When performing this first merge, we are losing almost 350k rows. The colonia/alcaldia names on both datasets might be different but refer to the same thing; hence, an homologation process should be done in order to keep the rows in the merged dataframe. Some cool algorithm, like **fuzzywuzzy**, might help to avoid the long, brute force approach.  

In [ ]:
print(np.sort(crimes_final.alcaldia.unique()), '\n')
print(np.sort(cams.alcaldia.unique()))

The *alcaldia* values in both original dataframes are the same, so the homologation is needed on the *colonia* columns. Since there are more rows in crime dataset, the *colonia* values in the cams dataset will be the ones being adjusted. Performing a right join will help identify which colonia names have to be changed 

A list of the problematic *colonia* names from the cams dataset will helpt to replace those values by a substitute name related to the crimes dataset.


In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz, process

def get_best_substitute(substitute_this, potential_substitutes):
    # Get the best match and its score using fuzzy matching
    best_match, score = process.extractOne(substitute_this, potential_substitutes)
    if score >= 90:  # You can adjust the threshold as needed
        return best_match
    else:
        return None

1. Detect which *colonia* names need to be changed.
2. Create a dictionary 'substitute_dict' that contains the recommended substitions.
3. Create a function *get_substitute* that replaces *colonia* values 

In order to detect which *colonia* columns need to be changed in *cams* dataset , a new merge from *crimes_final* to *cams* will be done. The rows containing null values will also contain the *colonia* value that needs to be homologated

In [ ]:
merged_cams = cams.merge(crimes_final, how= 'left', on=['alcaldia','colonia'])
print(merged_cams.shape)
print(merged_cams.isnull().sum())

consolidated_nulls= merged_cams[merged_cams["idcarpeta"].isnull()].copy()
print(consolidated_nulls.shape)

consolidated_nulls.head()


### Code to create a substitution dictionary 

This dictionary will contain *colonia* values from the *cams* dataset as keys, and a recommended substitution from a list of *colonias* from the *crimes_final* dataset

In [ ]:
#Get the unique pairs 'alcalduia'-'colonia'
missing_alcaldia_colonia_pairs = consolidated_nulls.drop_duplicates(subset=['alcaldia', 'colonia'])[['alcaldia', 'colonia']].values.tolist()
original_alcaldia_colonia_pairs = crimes_final.drop_duplicates(subset=['alcaldia', 'colonia'])[['alcaldia', 'colonia']].values.tolist()

#missing_colonias =list(consolidated_nulls.colonia.unique())
#substitute_list = list(crimes_final.colonia.unique())

#missing_colonias.sort()
#substitute_list.sort()
missing_alcaldia_colonia_pairs.sort()
original_alcaldia_colonia_pairs.sort()
print(len(missing_alcaldia_colonia_pairs))

#Getting the alcaldia - colonia pair inside a single string with '-' as separator 
missing_colonias  = [f"{pair[0]} - {pair[1]}" for pair in missing_alcaldia_colonia_pairs]
original_colonias = [f"{pair[0]} - {pair[1]}" for pair in original_alcaldia_colonia_pairs]

In [ ]:
# Create a dictionary with target values as keys and their best substitutes as values
substitute_dict = {value: get_best_substitute(value, original_colonias) for value in missing_colonias}

By using *json.dumps* function, we get a clear display of the new substitution dictionary

In [ ]:
import json
pretty_dict = json.dumps(substitute_dict, indent=4)

print(pretty_dict)
#print(substitute_dict.get("ACUEDUCTO DE GUADALUPE MODULAR"))

In [ ]:
del cams["colonia_new"]

In [ ]:
import re

# Create a new colum in cams dataset to contain the substitution colonia value, if there exists any. Otherwise, keep using the original 'colonia' value
def get_substitute(row):
    key       = row['alcaldia'] + ' - ' + row['colonia']
    returnval = substitute_dict.get(key, row['colonia'])
    
    if not returnval:
        return row["colonia"]
    
    #if returnval.replace(r'.*- ', '', regex=True) == row['alcaldia']:
    #For cases when the alcaldia name was the best match
    if re.sub(r'.*- ', '', returnval) in (row['alcaldia'], 'CUAJIMALPA'):
        return row['colonia']
    
    #if returnval is null, return original value
    return returnval or row["colonia"]
    
cams['colonia_new'] = cams.apply(get_substitute, axis=1)
cams['colonia_new'] = cams['colonia_new'].str.replace(r'.*- ', '', regex=True)
cams.head(15)

After the substitutions are completed, we have to check duplicates again: since different *colonia* values were homologated into a single value, there will be duplicated rows with different *totalinsta* values. These rows need to be consolidated or grouped in a single row andgetting the sum of the *totalinsta* column: the total number of installed cams

In [ ]:
duplicates = cams[['alcaldia','colonia_new','totalinsta']].duplicated(keep=False)
duplicated_rows=cams[duplicates]
print(duplicated_rows.shape)
duplicated_rows[['id','alcaldia','colonia_new','totalinsta','colonia_lat','colonia_lon']].sort_values(by=['alcaldia','colonia_new'])


In [ ]:
duplicates = cams[['alcaldia','colonia_new']].duplicated(keep=False)
duplicated_rows=cams[duplicates]
print(duplicated_rows.shape)
duplicated_rows[['id','alcaldia','colonia_new','totalinsta','colonia_lat','colonia_lon']].sort_values(by=['alcaldia','colonia_new'])

The two above duplicate analysis show two things: that the *totalinsta* column sum of the duplicated records can be done, but also that there are different values in *colonia_lat* and *colonia_lon*.  The approach to follow is to get the sum of the installed cams 'totalinsta' and not to take into consideration the colonia_lat and colonia_lon columns for now.|

In [ ]:
#Before making the aggregation check number of 'totalinsta' cams
print(cams.loc[cams['colonia_new']=='AGRÍCOLA PANTITLAN', 'totalinsta'])
print(cams.loc[cams['colonia_new']=='EL MIRADOR I(SAN LUCAS XOCHIMANCA)', 'totalinsta'])


In [ ]:
cams_temp = cams.copy()
del cams_temp["colonia"]
del cams_temp["id"]

columns_to_group=cams_temp.columns.drop(['totalinsta'])
columns_to_group=['alcaldia','colonia_new']

# Group by 'alcaldia' and 'colonia', and sum the 'totalinsta' values
#cams_final = cams_temp.groupby(['alcaldia', 'colonia_new','prioritari','geo_shape','colonia_lat','colonia_lon'], as_index=False).agg({'totalinsta': 'sum'})
cams_final = cams_temp.groupby(columns_to_group, as_index=False).agg({'totalinsta': 'sum'})

cams_final 

In [ ]:
#After making the aggregation check number of 'totalinsta' cams
print(cams_final.loc[cams_final['colonia_new']=='AGRÍCOLA PANTITLAN', 'totalinsta'])
print(cams_final.loc[cams_final['colonia_new']=='EL MIRADOR I(SAN LUCAS XOCHIMANCA)', 'totalinsta'])


In [ ]:
#cols = ['alcaldia', 'colonia_new',	'prioritari', 'geo_shape', 'colonia_lat', 'colonia_lon','totalinsta']
cols = ['alcaldia', 'colonia_new']

duplicates = cams_final[cols].duplicated(keep=False)
duplicated_rows=cams_final[duplicates]
duplicated_rows.sort_values(by=['alcaldia','colonia_new'])

In [ ]:
cams_final.isnull().sum()

In [ ]:
final_merge = crimes_final.merge(cams_final, how ='left', left_on=['alcaldia', 'colonia'], right_on=['alcaldia','colonia_new'])
print(final_merge.shape)
print(final_merge.isnull().sum())
final_merge.head()

In [ ]:
final_merge[final_merge.isnull().any(axis=1)]


In [ ]:
#Drop records with null values
crimes_cams_merge= final_merge.dropna().copy()
print(crimes_cams_merge.shape)
crimes_cams_merge['totalinsta'] = crimes_cams_merge['totalinsta'].round().astype(int)
crimes_cams_merge.head(5)

In [ ]:
crimes_cams_merge.to_csv("../datasets/crimes_cams.csv",index=False)


### Similar methods for creationf substitute dictionary

In [ ]:
def homologation(row, dict_subs, dict_compare, sub_col, ref_col, type):
    to_be_substituted =row[sub_col]
    reference_val = row[ref_col]
    #Get the colonia substitution name
    recommended_sub = dict_subs.get(to_be_substituted)
    #obtain alcaldia name related to that substitution name
    validate_sub = dict_compare.get(recommended_sub)
    if reference_val == validate_sub:
        if type=='colonia':
            return recommended_sub 
        else:
            return validate_sub
    else:
        return None
    
    
# Create a new column 'colonia2' in the cams DataFrame
cams['colonia2'] = None
cams['relatedCol']=None


# Iterate over each row in the DataFrame and apply the homologation function
for index, row in cams.iterrows():
    cams.at[index, 'colonia2']   = homologation(row, substitute_dict, crimes_colonia_to_alcaldia_dict, 'colonia', 'alcaldia','colonia')
    cams.at[index, 'relatedCol'] = homologation(row, substitute_dict, crimes_colonia_to_alcaldia_dict, 'colonia', 'alcaldia','other')

    #axis=1
cams['colonia2'] = cams.apply(homologation, \
                              dict_subs = substitute_dict, \
                              dict_compare = crimes_colonia_to_alcaldia_dict, \
                              sub_col= "colonia", \
                              ref_col= 'alcaldia'\
                             )

    

To ensure that the *colonia* substitute name is correct, a comparison with the *alcaldias* values will be made before replacing the values. A lookup dataframe from *crimes* dataset will be created in order to pair up *colonia* with *alcaldia*

In [ ]:
crime_locations= crimes.groupby(['fgj_colonia_registro','alcaldia_hechos'],as_index=False).agg(count=('idcarpeta','count')).copy()
crime_locations.head()


In [ ]:
homologation["alcaldia_cams"] = cams.loc[ (cams["colonia"] == homologation["cams_colonia"]), "alcaldia"]
homologation["alcaldia_crime"] = crime_locations.loc[crime_locations["fgj_colonia_registro"] == homologation["substitute_colonia"], \
                                                     "alcaldia_hechos"]
homologation["matches"] = (homologation["alcaldia_crime"] == homologation["alcaldia_cams"])

homologation.head()
#df.loc[df['Age'] > 25, 'Gender'] = 'M'



In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Function to find the best fuzzy match from 'c5_agg' for each value in 'consolidated'
def find_fuzzy_match(value, ref_df, ref_col):
    match, score, *_ = process.extractOne(value, ref_df[ref_col])
    if score >= 80:  # You can adjust the threshold as needed
        return match
    else:
        return None

In [ ]:
#df.loc[df['Age'] > 25, 'Gender'] = 'M'

pretty_dict[0]

In [ ]:
for old_colonia in pretty_dict.keys():
    alcaldia_old = cams[cams["colonia"] == alcaldia_old]
    if old_colonia

## Relation with *stations* dataframe

Something  more complex should be done in order to relate the new consolidated dataframe with the *stations* dataset. The way to do this is to calculate the distance between the crime locations and all the metro stations in order to find which is the nearest one. The **harvesine_distance** function will be applied and the new columns *nearest_distance*, *nearest_location* and *nearest_station* will be populated. 


In [ ]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    # Earth's radius in kilometers
    earth_radius = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Calculate differences in latitude and longitude
    d_lat = lat2_rad - lat1_rad
    d_lon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(d_lat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(d_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate the distance
    distance = earth_radius * c
    return distance


In [ ]:
def find_nearest_location(ref_lat, ref_lon, locations, stationsDf):
    nearest_distance = float('inf')
    nearest_location = None
    
   
    #Loop to compare crime location with each of the 195 metro station locations
    for lat, lon, station_name in locations:
        lat = float(lat)
        lon = float(lon)
        distance = haversine_distance(ref_lat, ref_lon, lat, lon)
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_location = (lat, lon)
            nearest_station  = station_name
    
    return nearest_distance, nearest_location, nearest_station


In [ ]:
crimes_cams_merge['nearest_distance'], crimes_cams_merge['nearest_location'], crimes_cams_merge['nearest_station'] = zip(*crimes_cams_merge.apply(
    lambda row: find_nearest_location( ref_lat   = row['crimen_lat'], 
                                       ref_lon   = row['crimen_lon'], 
                                       locations = stations[['station_lat', 'station_lon', 'nombre']].values,
                                       stationsDf = stations
                                     ),
                axis = 1
    )
  )

In [ ]:
crimes_cams_merge.head()




In [ ]:
print('Min. nearest distance is ',crimes_cams_merge.nearest_distance.min())
print('Max. nearest distance is ',crimes_cams_merge.nearest_distance.max())
crimes_cams_merge.nearest_station.unique()


In [ ]:
crimes_cams_merge.shape

This is the consolidated dataset that can be used as base for future calculations. It will be exported as a csv at this stage. However, more data related to camera locations related to program **C5** was found, so this info can be merged as well so that another dataset is used


In [ ]:
crimes_cams_merge.to_csv('../datasets/consolidated.csv.zip', compression = 'zip', index=False)

## C5 cameras
Mexico City's security organization **C5** installed Wifi Modules on their public city security cameras. The data related to those cameras and their locations is public and can be used on this project

In [ ]:
consolidated_path = "\datasets\consolidated.zip"

zf = zipfile.ZipFile(repo_path+consolidated_path) 
consolidated = pd.read_csv(zf.open('consolidated'))
consolidated.head()


In [ ]:
consolidated.isnull().sum()

In [ ]:
c5_path = "\datasets\c5_cams.zip"

zf = zipfile.ZipFile(repo_path+c5_path) 
c5_raw = pd.read_csv(zf.open('c5_cams.csv'), encoding = "latin-1")

c5_raw.head()

In [ ]:
c5_raw.rename(columns = lambda x : x.lower() , inplace = True)
c5_raw.rename(columns = {'alcaldía':'alcaldia'}, inplace = True)
del c5_raw["programa"]
del c5_raw["puntos_de_acceso"]


In [ ]:
c5_raw.head()

The next step is to somehow relate this information to the consolidated dataset. Some ideas are:

- Counting the number of cameras installed on each neighborhood (colonia) and/or alcaldia/delegacion.  Also, counting the number of cameras within a certain radius.
- Calculating which is the crime's nearest camera by using the coordinates of the crime and the camera and calculating nearest distance. The calculation would be done only for cameras in the same 'alcaldia' or neighborhood to reduce processing time
- A dictionary containing which alcaldias are near from each other, so that the above step also could consider the nearby alcaldias.


In [ ]:
consolidated.shape


In [ ]:
c5_agg = c5_raw.groupby('colonia').agg(count=('id','count'), alcaldia=('alcaldia', 'first'))
c5_agg.reset_index(inplace=True)
c5_agg.rename(columns = {'count':'c5_cam_col'}, inplace = True)
c5_agg.head()


In [ ]:
consolidated2 = pd.merge(consolidated, c5_agg[['colonia','c5_cam_col']], on='colonia', how='left')
#consolidated2["c5_cam_col"]= consolidated2["c5_cam_col"].round().astype(int)
consolidated2.head()

In [ ]:
#consolidated2["c5_cam_col"]= consolidated2["c5_cam_col"].round().astype(int)
consolidated2.isnull().sum()


There are a lot of null values in the c5_cam_col. The cause probably is some difference between the colonia names in *c5_agg* and *consolidated*. A homologation process could fix this. The colonia names from *c5_agg* which didn't find a match in *consolidated* dataframe must be identified:

In [ ]:
consolidated2[consolidated2["c5_cam_col"].isnull()].head()

In [ ]:
missing_colonias= consolidated2.loc[ consolidated2["c5_cam_col"].isnull(), 'colonia'].unique()

In [ ]:
l= list(missing_colonias)

In [ ]:
test= c5_agg[c5_agg["colonia"].str.contains('COPILCO',na=False, case=False)]
test

For this example, the substring *COPILCO* was used to search for similar colonia names. *COPILCO UNIVERSIDAD I.S.S.S.T.E.* on *c5_agg* is not an equal string to *COPILCO UNIVERSIDAD ISSSTE* from *consolidated2*, but in reality the values reference the same colonia. Since there are almost 300 values here, there should be a way to automate the process of finding the matching pairs of each colonia. A first approach is using an external library , such as **fuzzywuzzy**

In [ ]:
print(consolidated2[consolidated2["c5_cam_col"].isnull()].shape)
consolidated2[consolidated2["c5_cam_col"].isnull()].head()

An aggregated dataset from *consolidated* is created here so that the matching colonia name is searched among the unique values. 

In [ ]:
#c5_agg['possible_substitute'] = c5_agg['colonia'].apply(find_fuzzy_match, ref_df=consolidated, ref_col='colonia')

consolidated_agg = consolidated.groupby('colonia').agg(count=('colonia','count'), alcaldia=('alcaldia', 'first'))
consolidated_agg.reset_index(inplace=True)

consolidated_agg.head()

#print(find_fuzzy_match('COPILCO UNIVERSIDAD I.S.S.S.T.E.', consolidated, 'colonia'))

In [ ]:

# Function to find the best fuzzy match from 'consolidated' for each value in 'c5_agg'
#c5_agg['possible_substitute'] = c5_agg['colonia'].apply(find_fuzzy_match, ref_df=consolidated, ref_col='colonia')

# Find possible substitutes from df2 for each value in df1's 'col1'
c5_agg['possible_substitute'] = c5_agg['colonia'].map(lambda x: find_fuzzy_match(x, consolidated_agg, 'colonia'))



In [ ]:
homologated = c5_agg[c5_agg["colonia"]!= c5_agg["possible_substitute"]]

condition = consolidated_agg["colonia"] == homologated["possible_substitute"]
homologated["alcaldia_substitute"] = consolidated_agg[condition]["alcaldia"]
homologated.head()

In [ ]:
homologated.sort_values(by="colonia", inplace = True)
homologated

In [ ]:
pd.set_option('display.max_columns', None)
homologated = homologated.loc[:, ['colonia', 'possible_substitute','alcaldia']]
homologated.to_csv('homologated.csv')

In [ ]:
c5_agg[c5_agg['colonia'].str.contains("AGRICOLA")]

In [ ]:
#print(l)
consolidated_agg[consolidated_agg['colonia'].str.contains("AGRÍCOLA")]

In [ ]:
c5_agg[]